In [51]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [53]:
DATASET_DIR = "dataset/emotion"
OUTPUT_CSV = "fer2013_mp_landmarks.csv"

In [54]:
EMOTIONS = ['angry','disgust','fear','happy','neutral','sad','surprise']
emotion_map = {emo:i for i,emo in enumerate(EMOTIONS)}

mp_face = mp.solutions.face_mesh

In [55]:
def get_landmarks(image_path, img_size=48):
    img = cv2.imread(image_path)
    if img is None:
        return None

    # Convert BGR -> RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    with mp_face.FaceMesh(static_image_mode=True,
                          max_num_faces=1,
                          refine_landmarks=False,
                          min_detection_confidence=0.5) as face_mesh:
        result = face_mesh.process(img_rgb)

        if not result.multi_face_landmarks:
            return None

        landmarks = []
        for lm in result.multi_face_landmarks[0].landmark:
            # Scale x, y về kích thước ảnh
            landmarks.append(int(lm.x * img_size))
            landmarks.append(int(lm.y * img_size))

        return landmarks


In [56]:
data = []

for subset in ['train','test']:
    for emotion in EMOTIONS:
        folder = os.path.join(DATASET_DIR, subset, emotion)
        if not os.path.exists(folder):
            continue
        
        print("Processing:", subset, emotion)
        for img_name in tqdm(os.listdir(folder)):
            if not img_name.lower().endswith(('.png','.jpg','.jpeg')):
                continue
            
            img_path = os.path.join(folder, img_name)
            lms = get_landmarks(img_path, img_size=48)
            
            if lms is not None:
                row = [img_path, subset, emotion_map[emotion], emotion] + lms
                data.append(row)

df = pd.DataFrame(data)
df.to_csv(OUTPUT_CSV, index=False)
print("DONE! Saved:", OUTPUT_CSV)


Processing: train angry


100%|██████████| 3995/3995 [01:14<00:00, 53.52it/s]


Processing: train disgust


100%|██████████| 436/436 [00:08<00:00, 52.89it/s]


Processing: train fear


...

Processing: test sad


100%|██████████| 1247/1247 [00:25<00:00, 49.02it/s]


Processing: test surprise


100%|██████████| 831/831 [00:16<00:00, 51.56it/s]


DONE! Saved: fer2013_mp_landmarks.csv
